# ETL User Reviews

Importamos las librerias que utilizaremos en esta etapa del proyecto.

In [119]:
import pandas as pd
import numpy as np
import ast
import json

## Extracción de Datos:

Hemos comenzado descomprimiendo los archivos json.gz con WinRAR, haciendo que los datos estén listos para su procesamiento en VS Code. Luego lo cargo en un DataFrame para su uso posterior.

In [120]:
# Ruta al archivo JSON
raw_data = 'australian_user_reviews.json'

rows = []

# Probar con una codificación específica si utf-8 no funciona
with open(raw_data, encoding='MacRoman') as f:
    for line in f.readlines():
        try:
            rows.append(ast.literal_eval(line))
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_user_reviews = pd.DataFrame(rows)
print(df_user_reviews.head())


             user_id                                           user_url  \
0  76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1            js41637               http://steamcommunity.com/id/js41637   
2          evcentric             http://steamcommunity.com/id/evcentric   
3              doctr                 http://steamcommunity.com/id/doctr   
4          maplemage             http://steamcommunity.com/id/maplemage   

                                             reviews  
0  [{'funny': '', 'posted': 'Posted November 5, 2...  
1  [{'funny': '', 'posted': 'Posted June 24, 2014...  
2  [{'funny': '', 'posted': 'Posted February 3.',...  
3  [{'funny': '', 'posted': 'Posted October 14, 2...  
4  [{'funny': '3 people found this review funny',...  


In [121]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


## Transformación de Datos:

En segundo lugar, continuamos con la transformación. Esta consiste en limpiar, modificar y preparar los datos para su uso posterior. Eso incluye eliminar datos nulos, convertir tipos de datos, normalizar columnas, desanidar listas, etc.

### DATOS NULOS y COLUMNAS IRRELEVANTES

Ahora que se la magnitud del dataset empiezo a depurarlo. En primer lugar voy a cheuqear si hay o no **id** que tengan **valor nulo**, dado que eso me podria traer problemas al relacionar mis datos con los otros dataset. De existir **id** sin valor, entonces eliminaria esas filas.

Dada la informacion que entrega df_user_reviews.info() notamos que en este caso el DataFrame original  NO tiene valores nulos dado que la cantidad de filas se corresponden con la cantidad  de id, url y reviews. Por ende no se justifica averiguar filas con data nula ni rearmarla. Por esta razón pasamos directamente a la eliminación de columnas que puedan estar de más para nuesstro objetivo puntual del proyecto. 

In [122]:
columns_relevant = ['user_id', 'reviews']
df_user_reviews2 = df_user_reviews[columns_relevant]
df_user_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  25799 non-null  object
 1   reviews  25799 non-null  object
dtypes: object(2)
memory usage: 403.2+ KB


### CALIDAD DE DATOS

In [123]:
df_user_reviews2.dtypes

user_id    object
reviews    object
dtype: object

**user_id**: analizamos que tipos de datos tiene dado que está definido como object. AL ver tan solo algunos ejemplos de la tabla notamos que hay tanto nombres como números. Por ello no se justifica modifcarlos.

In [124]:
df_user_reviews2.head(10)

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
5,Wackky,"[{'funny': '', 'posted': 'Posted May 5, 2014.'..."
6,76561198079601835,"[{'funny': '1 person found this review funny',..."
7,MeaTCompany,"[{'funny': '', 'posted': 'Posted July 24.', 'l..."
8,76561198089393905,"[{'funny': '5 people found this review funny',..."
9,76561198156664158,"[{'funny': '', 'posted': 'Posted June 16.', 'l..."


**reviews**: En este caso, lo primero y más importante será desanidar los datos, dado que como se observa en la tabla de arriba ser diccionarios anidados en en cada dato de la columna.

In [125]:
def reviews_extract(df):
    
    reviews_list = []

    # Iterar sobre cada fila del DataFrame
    for _, row in df.iterrows():
        user_id = row['user_id']
        reviews = row['reviews']  

        # Iterar sobre cada reseña en la lista y extraemos la info necesaria 
        for review in reviews:
            item_id = review.get('item_id')
            recommend = review.get('recommend')
            review_text = review.get('review')
          
            reviews_list.append([user_id, item_id, recommend, review_text])

    # Crear un nuevo DataFrame
    df_reviews = pd.DataFrame(reviews_list, columns=['user_id', 'item_id', 'recommend', 'user_review'])
    return df_reviews

# Llamar a la función para extraer las reseñas
df_user_reviews3 = reviews_extract(df_user_reviews2)

# Ver las primeras filas del nuevo DataFrame
print(df_user_reviews3.head())


             user_id item_id  recommend  \
0  76561197970982479    1250       True   
1  76561197970982479   22200       True   
2  76561197970982479   43110       True   
3            js41637  251610       True   
4            js41637  227300       True   

                                         user_review  
0  Simple yet with great replayability. In my opi...  
1               It's unique and worth a playthrough.  
2  Great atmosphere. The gunplay can be a bit chu...  
3  I know what you think when you see this title ...  
4  For a simple (it's actually not all that simpl...  


In [126]:
# Convertir item_id a entero y recommend a booleano
df_user_reviews3['item_id'] = df_user_reviews3['item_id'].astype(int)
df_user_reviews3['recommend'] = df_user_reviews3['recommend'].astype(bool)

# Verificar los tipos de datos
print(df_user_reviews3.dtypes)

user_id        object
item_id         int64
recommend        bool
user_review    object
dtype: object


In [127]:
df_user_reviews3.head()

,user_id,item_id,recommend,user_review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,True,It's unique and worth a playthrough.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,True,I know what you think when you see this title ...
4,js41637,227300,True,For a simple (it's actually not all that simpl...


### Feature Engineering: Analisis de sentiminentos.
Partiendo de review, se debe clasificar en: 

**•	0 si es negativa**
**•	1 si es neutral o si la reseña está ausente**
**•	2 si es positiva.**


In [128]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Descargar el léxico VADER
nltk.download('vader_lexicon')

#Creación una instancia del analizador
analyzer = SentimentIntensityAnalyzer()

df_user_reviews3['sentiment_analysis'] = df_user_reviews3['user_review'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
# clasificar el sentimiento
df_user_reviews3['sentiment_analysis'] = df_user_reviews3['sentiment_analysis'].apply(lambda x: 0 if x < -0.3 else (1 if -0.3 <= x <= 0.3 else 2))
#df_user_reviews3['review_neg'] = df_user_reviews3['sentiment_analysis'].apply(lambda x: 1 if x< -0.3 else 0)
#df_user_reviews3['review_neu'] = df_user_reviews3['sentiment_analysis'].apply(lambda x: 1 if -0.3<=x <= 0.3 else 0)
#df_user_reviews3['review_pos'] = df_user_reviews3['sentiment_analysis'].apply(lambda x: 1 if x> 0.3 else 0)

#Eliminar columna user_reviews para reemplazar por sentimentalAnalysis
df_user_reviews3 = df_user_reviews3.drop('user_review', axis=1)
df_user_reviews3.head()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MegaTecnologia\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,1
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,2


### Cuidado Estético y Reorganización de columnas

In [129]:
# Normalizar nombres de columnas a español
new_column_names = {
    'user_id': 'id_usuario',
    'item_id': 'id_item',
    'recommend': 'recomendación',
     'sentiment_analysis' : 'analisis_de_sentimientos' 
}
df_user_reviews3 = df_user_reviews3.rename(columns=new_column_names)
df_user_reviews3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_usuario                59305 non-null  object
 1   id_item                   59305 non-null  int64 
 2   recomendación             59305 non-null  bool  
 3   analisis_de_sentimientos  59305 non-null  int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 1.4+ MB


In [130]:
# Eliminar filas duplicadas en caso de que existan
df_user_reviews4 = df_user_reviews3.drop_duplicates()

# Verificar el DataFrame después de eliminar duplicados, como se ve en el resultado no hay duplicados en esta altura.
print(df_user_reviews4.info())


<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 59304
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_usuario                58431 non-null  object
 1   id_item                   58431 non-null  int64 
 2   recomendación             58431 non-null  bool  
 3   analisis_de_sentimientos  58431 non-null  int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 1.8+ MB
None


## Carga de Datos (Load)

En esta fase final del proceso ETL, los datos limpios y transformados se cargan en un sistema de almacenamiento o base de datos de destino. En nuestro caso utilizaremos Parquet dada su eficiencia. Yquedan ya listos para ser usados por FastAPI y Render.

In [131]:
import pyarrow
import fastparquet

In [132]:
# Guardar el DataFrame en formato CSV
df_user_reviews4.to_csv('australian_user_reviews.csv', index=False)
print("Archivo CSV guardado exitosamente.")

# Guardar el DataFrame en formato Parquet
df_user_reviews4.to_parquet('australian_user_reviews.parquet', index=False, engine='pyarrow', compression='snappy')
print("Archivo Parquet guardado exitosamente.")

Archivo CSV guardado exitosamente.
Archivo Parquet guardado exitosamente.


**Muestra Aleatoria**

En ciertas ocasiones puede ser de gran utilidad una muestra más pequeña de nuestro dataset para agilizar el trabajo. Y para que sea representativa la haremos aleatoria

In [133]:
# Selección aleatoria de 30,000 filas
df_random_sample = df_user_reviews4.sample(n=30000, random_state=27)
print(df_random_sample.head())
print(df_random_sample.info())


              id_usuario  id_item  recomendación  analisis_de_sentimientos
25511           baranaka      730           True                         2
37806       imarockerbtw      570           True                         0
40980           TSGStosh     9480           True                         2
25817      bryceblacktop    22300           True                         2
17552  76561198046324554   228260           True                         2
<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 25511 to 41019
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_usuario                30000 non-null  object
 1   id_item                   30000 non-null  int64 
 2   recomendación             30000 non-null  bool  
 3   analisis_de_sentimientos  30000 non-null  int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 966.8+ KB
None


In [134]:
df_random_sample.to_parquet('random_sample_reviews.parquet', index=False, engine='pyarrow', compression='snappy')
print("Muestra aleatoria para consultas guardada exitosamente.")


Muestra aleatoria para consultas guardada exitosamente.


In [135]:
dfprueba = pd.read_parquet('random_sample_reviews.parquet')

# Mostrar los nombres de las columnas
print(dfprueba.columns)

Index(['id_usuario', 'id_item', 'recomendación', 'analisis_de_sentimientos'], dtype='object')
